In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
from typing import Dict, List, Tuple
import csv
from io import StringIO


# ALGORITHM SECTION 

class EnergySource:
    def __init__(self, source_id, source_type, max_capacity, available_hours, cost_per_kwh):
        self.id = source_id
        self.type = source_type
        self.max_capacity = max_capacity 
        self.available_hours = available_hours  
        self.cost_per_kwh = cost_per_kwh
    
    def is_available(self, hour):
        start, end = self.available_hours
        if start <= end:
            return start <= hour <= end
        else:  
            return hour >= start or hour <= end
    
    def __repr__(self):
        return f"{self.type}({self.id}): {self.max_capacity}kWh @ Rs.{self.cost_per_kwh}/kWh"


class SmartGridOptimizer:
    def __init__(self, energy_sources: List[EnergySource], demand_tolerance=0.10):
        self.sources = energy_sources
        self.demand_tolerance = demand_tolerance  
        self.results = {}
    
    def greedy_allocation(self, hour: int, district_demands: Dict[str, float]) -> Dict:
      
        # Get available sources at this hour, sorted by cost
        available = [s for s in self.sources if s.is_available(hour)]
        available.sort(key=lambda x: x.cost_per_kwh)
        
        total_demand = sum(district_demands.values())
        min_demand = total_demand * (1 - self.demand_tolerance)
        max_demand = total_demand * (1 + self.demand_tolerance)
        
        allocation = {source.id: 0 for source in self.sources}
        district_allocation = {district: {source.id: 0 for source in self.sources} 
                              for district in district_demands.keys()}
        
        total_allocated = 0
        
        # Allocate greedily
        for source in available:
            if total_allocated >= total_demand:
                break
            
            # How much can we allocate from this source?
            available_capacity = source.max_capacity
            needed = total_demand - total_allocated
            to_allocate = min(available_capacity, needed)
            
            # Distribute proportionally to districts
            for district, demand in district_demands.items():
                proportion = demand / total_demand
                district_amount = to_allocate * proportion
                district_allocation[district][source.id] = district_amount
                allocation[source.id] += district_amount
            
            total_allocated += to_allocate
        
        # Calculate metrics
        total_cost = sum(allocation[s.id] * s.cost_per_kwh for s in self.sources)
        percent_met = (total_allocated / total_demand * 100) if total_demand > 0 else 0
        
        # Check if demand is satisfied within tolerance
        satisfied = min_demand <= total_allocated <= max_demand or total_allocated >= total_demand
        
        return {
            'hour': hour,
            'allocation': allocation,
            'district_allocation': district_allocation,
            'total_allocated': total_allocated,
            'total_demand': total_demand,
            'total_cost': total_cost,
            'percent_met': percent_met,
            'satisfied': satisfied,
            'method': 'Greedy'
        }
    
    def dynamic_programming_allocation(self, hour: int, district_demands: Dict[str, float]) -> Dict:
        """
        Dynamic Programming approach for optimal allocation
        
        DP State: dp[i][energy] = minimum cost to allocate 'energy' kWh using first i sources
        
        This is a variant of the unbounded knapsack problem.
        """
        available = [s for s in self.sources if s.is_available(hour)]
        
        if not available:
            return self.greedy_allocation(hour, district_demands)
        
        total_demand = sum(district_demands.values())
        
        # For simplicity with DP, we'll discretize the energy values
        # Scale to integers (multiply by 10 to handle decimals)
        scale = 10
        target = int(total_demand * scale)
        max_capacity_total = int(sum(s.max_capacity for s in available) * scale)
        
        # DP array: dp[energy] = (min_cost, source_usage)
        dp = [float('inf')] * (max_capacity_total + 1)
        dp[0] = 0
        parent = [None] * (max_capacity_total + 1)
        
        # Fill DP table
        for energy in range(target + 1):
            if dp[energy] == float('inf'):
                continue
            
            for source in available:
                capacity_scaled = int(source.max_capacity * scale)
                
                for amount in range(1, min(capacity_scaled + 1, max_capacity_total - energy + 1)):
                    new_energy = energy + amount
                    if new_energy <= max_capacity_total:
                        cost = (amount / scale) * source.cost_per_kwh
                        new_cost = dp[energy] + cost
                        
                        if new_cost < dp[new_energy]:
                            dp[new_energy] = new_cost
                            parent[new_energy] = (energy, source.id, amount / scale)
        
        # Find best solution within tolerance
        min_target = int(total_demand * (1 - self.demand_tolerance) * scale)
        max_target = int(total_demand * (1 + self.demand_tolerance) * scale)
        
        best_energy = target
        best_cost = dp[target] if target <= max_capacity_total else float('inf')
        
        for e in range(min_target, min(max_target + 1, max_capacity_total + 1)):
            if dp[e] < best_cost:
                best_cost = dp[e]
                best_energy = e
        
        # Reconstruct solution
        allocation = {source.id: 0 for source in self.sources}
        current = best_energy
        
        while current > 0 and parent[current]:
            prev_energy, source_id, amount = parent[current]
            allocation[source_id] += amount
            current = prev_energy
        
        # Distribute to districts proportionally
        district_allocation = {district: {source.id: 0 for source in self.sources} 
                              for district in district_demands.keys()}
        
        total_allocated = sum(allocation.values())
        for district, demand in district_demands.items():
            proportion = demand / total_demand if total_demand > 0 else 0
            for source_id, amount in allocation.items():
                district_allocation[district][source_id] = amount * proportion
        
        percent_met = (total_allocated / total_demand * 100) if total_demand > 0 else 0
        
        return {
            'hour': hour,
            'allocation': allocation,
            'district_allocation': district_allocation,
            'total_allocated': total_allocated,
            'total_demand': total_demand,
            'total_cost': best_cost,
            'percent_met': percent_met,
            'satisfied': True,
            'method': 'Dynamic Programming'
        }
    
    def optimize_full_day(self, hourly_demands: Dict[int, Dict[str, float]], use_dp=False):
        """Optimize energy allocation for all hours"""
        results = []
        
        for hour, demands in sorted(hourly_demands.items()):
            if use_dp:
                result = self.dynamic_programming_allocation(hour, demands)
            else:
                result = self.greedy_allocation(hour, demands)
            results.append(result)
        
        return results
    
    def generate_report(self, results: List[Dict]) -> str:
        """Generate comprehensive report"""
        report = []
        report.append("=" * 80)
        report.append("SMART ENERGY GRID OPTIMIZATION REPORT - NEPAL")
        report.append("=" * 80)
        report.append("")
        
        total_cost = sum(r['total_cost'] for r in results)
        total_energy = sum(r['total_allocated'] for r in results)
        
        # Calculate renewable percentage
        renewable_sources = [s.id for s in self.sources if s.type.lower() in ['solar', 'hydro']]
        renewable_energy = sum(
            sum(r['allocation'].get(sid, 0) for sid in renewable_sources)
            for r in results
        )
        renewable_percent = (renewable_energy / total_energy * 100) if total_energy > 0 else 0
        
        # Summary
        report.append(f"Total Cost of Distribution: Rs. {total_cost:.2f}")
        report.append(f"Total Energy Distributed: {total_energy:.2f} kWh")
        report.append(f"Renewable Energy Percentage: {renewable_percent:.2f}%")
        report.append("")
        
        # Diesel usage analysis
        diesel_sources = [s.id for s in self.sources if s.type.lower() == 'diesel']
        diesel_hours = []
        
        for r in results:
            diesel_used = sum(r['allocation'].get(sid, 0) for sid in diesel_sources)
            if diesel_used > 0:
                diesel_hours.append(f"Hour {r['hour']:02d}: {diesel_used:.2f} kWh")
        
        if diesel_hours:
            report.append("Diesel Usage (Expensive Source):")
            for entry in diesel_hours:
                report.append(f"  • {entry}")
            report.append(f"  Reason: Other sources at capacity or unavailable")
        else:
            report.append("✓ No diesel usage - 100% renewable energy!")
        
        report.append("")
        report.append("Algorithm Efficiency:")
        method = results[0]['method'] if results else 'Unknown'
        report.append(f"  Method Used: {method}")
        
        if method == 'Greedy':
            report.append("  • Fast computation (O(n log n) per hour)")
            report.append("  • Always chooses cheapest available sources")
            report.append("  • May not be globally optimal but very efficient")
        else:
            report.append("  • Optimal solution using Dynamic Programming")
            report.append("  • Guarantees minimum cost allocation")
            report.append("  • Higher computation time but best cost")
        
        return "\n".join(report)


#GUI SECTION

class SmartGridGUI:
    """GUI for Smart Energy Grid Optimizer"""
    
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Energy Grid - Nepal")
        self.root.geometry("1200x800")
        self.root.configure(bg="#0a0e27")
        
        # Default data
        self.energy_sources = [
            EnergySource("S1", "Solar", 50, (6, 18), 1.0),
            EnergySource("S2", "Hydro", 40, (0, 23), 1.5),
            EnergySource("S3", "Diesel", 60, (17, 23), 3.0)
        ]
        
        self.hourly_demands = {
            6: {"District A": 20, "District B": 15, "District C": 25},
            7: {"District A": 22, "District B": 16, "District C": 28},
        }
        
        self.results = []
        
        self.setup_ui()
    
    def setup_ui(self):
        
        # Title
        title_frame = tk.Frame(self.root, bg="#0a0e27")
        title_frame.pack(pady=15)
        
        tk.Label(
            title_frame,
            text="Smart Energy Grid Optimizer",
            font=("Arial", 28, "bold"),
            bg="#0a0e27",
            fg="#00ff41"
        ).pack()
        
        tk.Label(
            title_frame,
            text="Load Distribution Optimization for Nepal",
            font=("Arial", 12),
            bg="#0a0e27",
            fg="#888888"
        ).pack()
        
        # Main container
        main_container = tk.Frame(self.root, bg="#0a0e27")
        main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Input
        left_panel = tk.Frame(main_container, bg="#1a1f3a", relief=tk.RAISED, bd=2)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 5))
        
        self.setup_input_panel(left_panel)
        
        # Right panel - Output
        right_panel = tk.Frame(main_container, bg="#1a1f3a", relief=tk.RAISED, bd=2)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=(5, 0))
        
        self.setup_output_panel(right_panel)
    
    def setup_input_panel(self, parent):
        
        tk.Label(
            parent,
            text="Input Data",
            font=("Arial", 16, "bold"),
            bg="#1a1f3a",
            fg="#00ff41"
        ).pack(pady=10)
        
        # Energy Sources Section
        sources_frame = tk.LabelFrame(
            parent,
            text="Energy Sources",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="white",
            relief=tk.GROOVE,
            bd=2
        )
        sources_frame.pack(pady=5, padx=10, fill=tk.BOTH)
        
        self.sources_tree = ttk.Treeview(
            sources_frame,
            columns=("ID", "Type", "Capacity", "Hours", "Cost"),
            show="headings",
            height=3
        )
        
        self.sources_tree.heading("ID", text="ID")
        self.sources_tree.heading("Type", text="Type")
        self.sources_tree.heading("Capacity", text="Max kWh")
        self.sources_tree.heading("Hours", text="Available")
        self.sources_tree.heading("Cost", text="Rs/kWh")
        
        self.sources_tree.column("ID", width=40)
        self.sources_tree.column("Type", width=70)
        self.sources_tree.column("Capacity", width=70)
        self.sources_tree.column("Hours", width=80)
        self.sources_tree.column("Cost", width=60)
        
        self.sources_tree.pack(pady=5, padx=5, fill=tk.BOTH)
        self.populate_sources_table()
        
        # Demand Section
        demand_frame = tk.LabelFrame(
            parent,
            text="Hourly Demand (kWh)",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="white",
            relief=tk.GROOVE,
            bd=2
        )
        demand_frame.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        # Demand input
        tk.Label(
            demand_frame,
            text="Enter demands (format: Hour,DistA,DistB,DistC per line):",
            bg="#1a1f3a",
            fg="white",
            font=("Arial", 9)
        ).pack(pady=5)
        
        self.demand_text = scrolledtext.ScrolledText(
            demand_frame,
            height=8,
            bg="#0f1626",
            fg="#00ff41",
            font=("Courier", 10),
            insertbackground="white"
        )
        self.demand_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)
        self.demand_text.insert("1.0", "6,20,15,25\n7,22,16,28\n")
        
        # Algorithm Selection
        algo_frame = tk.Frame(parent, bg="#1a1f3a")
        algo_frame.pack(pady=10)
        
        tk.Label(
            algo_frame,
            text="Algorithm:",
            bg="#1a1f3a",
            fg="white",
            font=("Arial", 11, "bold")
        ).pack(side=tk.LEFT, padx=5)
        
        self.algorithm_var = tk.StringVar(value="greedy")
        
        tk.Radiobutton(
            algo_frame,
            text="Greedy",
            variable=self.algorithm_var,
            value="greedy",
            bg="#1a1f3a",
            fg="white",
            selectcolor="#0a0e27",
            font=("Arial", 10)
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Radiobutton(
            algo_frame,
            text="Dynamic Programming",
            variable=self.algorithm_var,
            value="dp",
            bg="#1a1f3a",
            fg="white",
            selectcolor="#0a0e27",
            font=("Arial", 10)
        ).pack(side=tk.LEFT, padx=5)
        
        # Buttons
        button_frame = tk.Frame(parent, bg="#1a1f3a")
        button_frame.pack(pady=10)
        
        tk.Button(
            button_frame,
            text="Optimize",
            font=("Arial", 12, "bold"),
            bg="#00ff41",
            fg="black",
            width=15,
            command=self.run_optimization,
            cursor="hand2"
        ).grid(row=0, column=0, padx=5)
        
        tk.Button(
            button_frame,
            text="Clear",
            font=("Arial", 12, "bold"),
            bg="#ff4757",
            fg="white",
            width=15,
            command=self.clear_all,
            cursor="hand2"
        ).grid(row=0, column=1, padx=5)
    
    def setup_output_panel(self, parent):
        
        tk.Label(
            parent,
            text="Results & Analysis",
            font=("Arial", 16, "bold"),
            bg="#1a1f3a",
            fg="#00ff41"
        ).pack(pady=10)
        
        # Summary
        summary_frame = tk.LabelFrame(
            parent,
            text="Summary",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="white",
            relief=tk.GROOVE,
            bd=2
        )
        summary_frame.pack(pady=5, padx=10, fill=tk.X)
        
        self.summary_label = tk.Label(
            summary_frame,
            text="Total Cost: --- | Renewable: ---%",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="#ffd700"
        )
        self.summary_label.pack(pady=10)
        
        # Results Table
        results_frame = tk.LabelFrame(
            parent,
            text="Hourly Allocation",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="white",
            relief=tk.GROOVE,
            bd=2
        )
        results_frame.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        # Create Treeview for results
        columns = ("Hour", "Solar", "Hydro", "Diesel", "Total", "Demand", "Met%")
        self.results_tree = ttk.Treeview(
            results_frame,
            columns=columns,
            show="headings",
            height=8
        )
        
        for col in columns:
            self.results_tree.heading(col, text=col)
            self.results_tree.column(col, width=80)
        
        scrollbar = ttk.Scrollbar(results_frame, orient=tk.VERTICAL, command=self.results_tree.yview)
        self.results_tree.configure(yscrollcommand=scrollbar.set)
        
        self.results_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, pady=5, padx=5)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Report
        report_frame = tk.LabelFrame(
            parent,
            text="Detailed Report",
            font=("Arial", 12, "bold"),
            bg="#1a1f3a",
            fg="white",
            relief=tk.GROOVE,
            bd=2
        )
        report_frame.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        self.report_text = scrolledtext.ScrolledText(
            report_frame,
            height=12,
            bg="#0f1626",
            fg="#00ff41",
            font=("Courier", 9),
            wrap=tk.WORD
        )
        self.report_text.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)
    
    def populate_sources_table(self):
        for item in self.sources_tree.get_children():
            self.sources_tree.delete(item)
        
        for source in self.energy_sources:
            hours = f"{source.available_hours[0]:02d}-{source.available_hours[1]:02d}"
            self.sources_tree.insert("", tk.END, values=(
                source.id,
                source.type,
                source.max_capacity,
                hours,
                source.cost_per_kwh
            ))
    
    def parse_demand_input(self):
        text = self.demand_text.get("1.0", tk.END).strip()
        demands = {}
        
        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue
            
            parts = [x.strip() for x in line.split(',')]
            if len(parts) != 4:
                continue
            
            try:
                hour = int(parts[0])
                demands[hour] = {
                    "District A": float(parts[1]),
                    "District B": float(parts[2]),
                    "District C": float(parts[3])
                }
            except ValueError:
                continue
        
        return demands
    
    def run_optimization(self):
        try:
            # Parse demands
            self.hourly_demands = self.parse_demand_input()
            
            if not self.hourly_demands:
                messagebox.showerror("Error", "No valid demand data entered!")
                return
            
            # Create optimizer
            optimizer = SmartGridOptimizer(self.energy_sources)
            
            # Run optimization
            use_dp = (self.algorithm_var.get() == "dp")
            self.results = optimizer.optimize_full_day(self.hourly_demands, use_dp=use_dp)
            
            # Display results
            self.display_results()
            
            # Generate report
            report = optimizer.generate_report(self.results)
            self.report_text.delete("1.0", tk.END)
            self.report_text.insert("1.0", report)
            
            messagebox.showinfo("Success", "Optimization completed!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Optimization failed: {str(e)}")
    
    def display_results(self):
        """Display results in table"""
        # Clear previous results
        for item in self.results_tree.get_children():
            self.results_tree.delete(item)
        
        total_cost = 0
        total_energy = 0
        renewable_energy = 0
        
        for result in self.results:
            hour = result['hour']
            solar = result['allocation'].get('S1', 0)
            hydro = result['allocation'].get('S2', 0)
            diesel = result['allocation'].get('S3', 0)
            total = result['total_allocated']
            demand = result['total_demand']
            met = result['percent_met']
            
            self.results_tree.insert("", tk.END, values=(
                f"{hour:02d}:00",
                f"{solar:.1f}",
                f"{hydro:.1f}",
                f"{diesel:.1f}",
                f"{total:.1f}",
                f"{demand:.1f}",
                f"{met:.0f}%"
            ))
            
            total_cost += result['total_cost']
            total_energy += total
            renewable_energy += solar + hydro
        
        # Update summary
        renewable_percent = (renewable_energy / total_energy * 100) if total_energy > 0 else 0
        self.summary_label.config(
            text=f"Total Cost: Rs. {total_cost:.2f} | Renewable: {renewable_percent:.1f}%"
        )
    
    def clear_all(self):
        self.demand_text.delete("1.0", tk.END)
        self.demand_text.insert("1.0", "6,20,15,25\n7,22,16,28\n")
        
        for item in self.results_tree.get_children():
            self.results_tree.delete(item)
        
        self.report_text.delete("1.0", tk.END)
        self.summary_label.config(text="Total Cost: --- | Renewable: ---%")
        self.results = []



if __name__ == "__main__":
    root = tk.Tk()
    app = SmartGridGUI(root)
    root.mainloop()